In [1]:
import time
import heapq
import numpy as np

from random import choice
from random import randint

from collections import Counter
from collections import defaultdict

In [2]:
#with open("Downloads/test1_new.txt") as f:
#    lines1 = f.read().splitlines()

with open("tests/test2_new.txt") as f:
    lines2 = f.read().splitlines()

with open("tests/test3_new.txt") as f:
    lines3 = f.read().splitlines()
    
with open("tests/test1_aycan.txt") as f:
    lines4 = f.read().splitlines() 
    
with open("tests/test_aycan.txt") as f:
    lines5 = f.read().splitlines() 

# Load File

In [3]:
lines=lines3

number_of_vertices = int(lines[0])
number_of_edges = int(lines[1])

vertices = lines[2:2+number_of_vertices]
edges = lines[2+number_of_vertices:]

ids_and_populations = [tuple(map(int, vertices[i].split(" "))) for i in range(len(vertices))]
populations = dict(sorted(dict(ids_and_populations).items())) #redundant sort

mydict = lambda: defaultdict(lambda: defaultdict())
G = mydict()

for i in range(len(edges)):
    source, target, weight = map(int, edges[i].split(" "))
    G[source][target] = weight
    G[target][source] = weight
    

# Functions

In [4]:
def select_neighbors(G, sub_graph, current_node, k):
    if k == 0:
        return sub_graph

    for j in G[current_node].items():
        sub_graph[current_node][j[0]] = j[1]
        sub_graph[j[0]][current_node] = j[1]

        sub_graph = select_neighbors(G, sub_graph, j[0], k - 1)
    return sub_graph


def merge_graph(dict1, dict2):
    for key, value in dict2.items():
        for subkey, subvalue in value.items():
            dict1[key][subkey] = subvalue
            dict1[subkey][key] = subvalue

# INPUT:
# G: dict, populations: dict
def dijkstra(G, populations, source): #FIXME: THIS IS POPULATION_LIST OF OTHERS
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0

    pq = []
    for node in G:
        heapq.heappush(pq, (node, costs[node]))

    while len(pq) != 0:
        current_node, current_node_distance = heapq.heappop(pq)
        for neighbor_node in G[current_node]:
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                costs[neighbor_node] = distance
                heapq.heappush(pq, (neighbor_node, distance))
    
    #return (costs.values(),population_list[])
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    populations_values_lst = list(dict(sorted(populations.items())).values())
    return np.sum(np.array(sorted_costs_lst) * np.array(populations_values_lst))


def random_start(G):
    res = [choice(list(G.keys())), choice(list(G.keys()))]
    if res[0] == res [1]:
        return random_start(G)
    print(f"Random start: {res}")
    return res


#//2 * O((V+E)*logV) = O(E*logV) //
def allocation_cost(G, population_list, i,j):
    return [dijkstra(G,population_list, i),dijkstra(G,population_list, j)]


# 2*O(V)*O(E*logV) = O(E*V*logV) #
def algo(G):
    population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
    #population_list = [populations[i] for i in G.keys()]
    selected_vertices = random_start(G) 
    selected_costs = allocation_cost(G,population_list, selected_vertices[0],selected_vertices[1])
    
    for not_selected in G.keys():
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra(G,population_list, not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                selected_vertices[bigger_index] = not_selected
    return(selected_vertices,selected_costs)


# V times Dijkstra
def sub_graph_apsp(G):
    population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
    selected_vertex = choice(list(G.keys()))
    selected_cost = dijkstra(G,population_list, selected_vertex)
    
    for node in G.keys():
        if node is not selected_vertex:
            this_cost = dijkstra(G, population_list, node) 
            if this_cost < selected_cost:
                selected_cost = this_cost
                selected_vertex = node    
    return selected_vertex, selected_cost


def algorithm_sub_graph_apsp(G, starting_node, k, hop_list):
    sub_graph = lambda: defaultdict(lambda: defaultdict())
    sub_graph = sub_graph()
    sub_graph = select_neighbors(G, sub_graph, current_node=starting_node, k=k)
    next_node, cost = sub_graph_apsp(sub_graph)
    #print(next_node)
    
    if len(hop_list) > 0 and next_node == hop_list[-1][0]:
        return next_node, cost
    
    hop_list.append((next_node, cost))
    return algorithm_sub_graph_apsp(G, next_node, k, hop_list)


def algo2(G, k):
    nodes = []
    costs = []
    
    for i in range(10):
        node, cost = algorithm_sub_graph_apsp(G, choice(list(G.keys())), k, [])
        nodes.append(node)
        costs.append(cost)
        print(node, cost)
        
    counter = Counter(nodes)
    most_commons = counter.most_common(2)
    target_nodes = (most_commons[0][0], most_commons[1][0])
    print(f"Target Nodes:", target_nodes)
    
    sub_graph1 = lambda: defaultdict(lambda: defaultdict())
    sub_graph1 = sub_graph1()
    sub_graph1 = select_neighbors(G, sub_graph1, target_nodes[0], k=k+1)
    
    sub_graph2 = lambda: defaultdict(lambda: defaultdict())
    sub_graph2 = sub_graph2()
    sub_graph2 = select_neighbors(G, sub_graph2, target_nodes[1], k=k+1)

    merge_graph(sub_graph1, sub_graph2)

    return algo(sub_graph1)
    #return target_nodes


In [ ]:
algo2(G, k=3)

495437 52074145778514
16024 46695469623712


In [ ]:

#for i in range(20):
#print(algo2(G, k=5))

# ([31470, 715224], [195627566180449, 194597112873142])


In [ ]:
#for i in range(10):
   # print(algo(G))

In [ ]:
#population_list = [populations[i] for i in G.keys()]
#dijkstra(G, population_list, 71)

In [ ]:
'''
sub_graph = lambda: defaultdict(lambda: defaultdict())
sub_graph = sub_graph()
start_time = time.time()
select_neighbors(G, sub_graph, 5000, k=7)
print(time.time() - start_time)

n = len(sub_graph.values()) / 2
n * (n-1) / 2
'''

# Main

In [ ]:
# Initialize the sub-graph
a = dict()
cur_node = 40
for i in range(10):
    sub_graph = lambda: defaultdict(lambda: defaultdict())
    sub_graph = sub_graph()
    sub_graph = select_neighbors(G, sub_graph, current_node=i*4 + 49, k=5)
    hospital_locations, costs = algo(sub_graph)
    
    if hospital_locations[0] not in a:
        a[hospital_locations[0]] = 1
    else:
        a[hospital_locations[0]] += 1
    if hospital_locations[1] not in a:
        a[hospital_locations[1]] = 1
    else:
        a[hospital_locations[1]] += 1
#print(hospital_locations, costs)

keys = np.array([k for k, v in a.items()])
vals = np.array([v for k, v in a.items()])

keys[np.argmax(vals)]

n = 2
indices = (-vals).argsort()[:n]
print(keys[indices[0]])
print(keys[indices[1]])

In [ ]:
#sub_graph = lambda: defaultdict(lambda: defaultdict())
#sub_graph = sub_graph()
#sub_graph = select_neighbors(G, sub_graph, current_node=40, k=4)
#hospital_locations, costs = algo(sub_graph)
#print(hospital_locations, costs)




In [ ]:
#sub_graph = lambda: defaultdict(lambda: defaultdict())
#sub_graph = sub_graph()
#sub_graph = select_neighbors(G, sub_graph, current_node=40, k=5)

#len(list(sub_graph.values()))

In [ ]:
#hospital_locations, costs = algo(G)
#print(hospital_locations, costs)

In [ ]:
#population_list = [populations[i] for i in G.keys()]
#dijkstra(G, population_list, 306690)

In [ ]:
from collections import Counter
a = [1,2,3,1,2,1,1,1,3,2,2,1]
b = Counter(a)
print(b.most_common(2))

# New Heuristic w/ added Local Optimization over 'algo'

In [ ]:
# Global/Regional Interchange Algorithm
def GRIA(G,k):
    population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
    selected_vertices = random_start(G) 
    selected_costs = allocation_cost(G,population_list, selected_vertices[0],selected_vertices[1])
    
    for not_selected in G.keys():
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra(G,population_list, not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                
                temp = regional_interchange(G,not_selected,k)
                if temp != min(selected_vertices):
                    selected_vertices[bigger_index] = temp
                # Regional Interchange
                # k-th neighbor -> 5. order neighbor subgraph
                    # V^2*logV 1-hospital problem
    return(selected_vertices,selected_costs)

def regional_interchange(G,current_node,k):
    #k-th neighbor subgraph
    sub_graph = lambda: defaultdict(lambda: defaultdict())
    sub_graph = sub_graph()
    select_neighbors(G, sub_graph, current_node, k)
    #1-hospital
    print(current_node,sub_graph_apsp(sub_graph)[0])
    return sub_graph_apsp(sub_graph)[0]
    

In [ ]:
GRIA(G,5)

In [ ]:
algo2(G,5)

In [ ]:
algo(G)

In [ ]:
def dijkstra_path(G, population_list, source):
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0
    #display(source,costs)

    pq = []
    for node in G:
        heapq.heappush(pq, (node, costs[node]))

    while len(pq) != 0:
        current_node, current_node_distance = heapq.heappop(pq)
        for neighbor_node in G[current_node]:
            #print(current_node,costs[source])
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                #if source==neighbor_node:
                    #print('here')
                costs[neighbor_node] = distance
                heapq.heappush(pq, (neighbor_node, distance))
    
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    sorted_populations_lst = list(dict(sorted(population_list.items())).values())
    #print(np.array(sorted_costs_lst) ,np.array(sorted_populations_lst))
    return np.array(sorted_costs_lst) * np.array(sorted_populations_lst)
    #return list(dict(sorted(costs.items())).values())

In [ ]:
population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
dijkstra_path(G, population_list, 15)

In [ ]:
population_list = [populations[i] for i in G.keys()]
#temp = dijkstra_path(G, population_list,200)
#[e[1] for e in sorted(list(temp.items()))]
#sorted(list(temp.items()))
G.keys()

In [ ]:
#APSP matrix
#def dijkstra(G, population_list, source):


APSP = np.zeros((number_of_vertices,number_of_vertices))
population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
for vertex in vertices:
    vertex= int(vertex.split()[0])
    APSP[vertex] = [e for e in dijkstra_path(G, population_list,vertex)]

def V4(APSP):
    n = len(APSP)

    res = {}

    temp_arr = APSP.copy()

    count=0
    for first in range(n):
        for second in range(n):
            if first==second:
                continue
            count+=1
            #print(first,second)
            temp_arr = APSP.copy()
            for row in temp_arr:
                if row[first]<row[second]:
                    row[second]=0
                else:
                    row[first]=0 
            #print(temp_arr,count)
            temp = sum(sum(temp_arr)) 
            #if temp in res.keys():
                #print(res[sum(sum(temp_arr))])
                #print('here!!!!!!!! ' ,first,second)
            #print(temp,(first,second))
            res[temp]=(first,second) #FIXME: there are collisions but is it a problem?

            

    ret=res[min(res.keys())]
    display(len(res))
    #display(res)
    print('pick {}th and {}th vertices to place hospitals!'.format(ret[0],ret[1]))
    return ret

#print(APSP)
V4(APSP)


In [ ]:
APSP = np.zeros((2,2))
APSP[0]

In [ ]:
# 8 13
# 9 16

display(sum(dijkstra_path(G, population_list, 8))+sum(dijkstra_path(G, population_list, 13)))
display(sum(dijkstra_path(G, population_list, 14))+sum(dijkstra_path(G, population_list, 16)))